In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import sklearn.cross_validation
from sklearn.cross_validation import cross_val_score

c:\users\augusto joaquin\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\users\augusto joaquin\appdata\local\programs\python\python35\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
c:\users\augusto joaquin\appdata\local\programs\python\python35\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and

In [2]:
def setPred(col):
    if(col > 0):
        return 1
    else:
        return 0
    
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

# preparar/tweks datos 

In [230]:
features = pd.read_csv("features.csv",low_memory = False)
prediccion = pd.read_csv("csv/labels_training_set.csv")

In [231]:
#features = features.loc[features["conversion"]<=5]

In [232]:
#mergeo features y la info q nos tiran, lleno con 0 los q no se
train = pd.merge(prediccion,features,on="person",how= "left").dropna()

In [233]:
prediccion = train["label"]

In [234]:
test = features.loc[~features["person"].isin(train["person"])]

In [235]:
listaDeCategoricos = ["condition_no_convercion","model_no_convercion","color_no_convercion","search_engine"\
                      ,"nombre mas usado dia","ultimo_dia_de_la_semana_activo","ultimo_evento_registrado_cmode",\
                  ]
for word in listaDeCategoricos:
    train[word],test[word] =  target_encode(trn_series = train[word],  \
                                            tst_series = test[word],\
                                            target=prediccion, \
                                            min_samples_leaf=10,\
                                            smoothing=0.1,\
                                            noise_level=0.00)


c:\users\augusto joaquin\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [236]:
train.columns

Index(['person', 'label', 'ad campaign hit', 'brand listing', 'checkout',
       'conversion', 'generic listing', 'lead', 'search engine hit',
       'searched products', 'staticpage', 'viewed product', 'visited site',
       'model_no_convercion', 'storage_no_convercion',
       'condition_no_convercion', 'color_no_convercion', 'search_engine',
       'numero mas usado dia', 'nombre mas usado dia',
       'ultimo_dia_de_la_semana_activo', 'ultimo_evento_registrado_cmode',
       'dias_hasta', 'viewed product_last', 'searched products_last',
       'ad campaign hit_last', 'staticpage_last', 'checkout_last',
       'search engine hit_last', 'conversion_last', 'generic listing_last',
       'brand listing_last', 'visited site_last', 'lead_last',
       'cantidad_de_dias_utilizado', 'media_dia_de_la_semana_no_convercion',
       'delta_ultimo_primer_dia_actividad',
       'delta_ultimo_anteulit_tiempo_actividad',
       'delta_primer_seg_tiempo_actividad', 'dTimeEveMax', 'dTimeEveMean0',


In [237]:
exclude = ["ultimo_dia_de_la_semana_activo","cantidad_de_dias_utilizado", \
           "media_dia_de_la_semana_no_convercion",  "storage_no_convercion",\
            "Estado Nuevo","generic listing", "Estado Excelente",\
           "visited site","dias_hasta","staticpage",\
          "dTimeEveMean0","dHTimeEveMean0","dTimeEveMax","search engine hit_last","search engine hit","searched products_last",]
train = train.drop(exclude,axis = 1)
test = test.drop(exclude,axis = 1)
#

In [238]:
prediccion.shape

(19126,)

In [239]:
train = train.drop(["person","label"],axis = 1)

In [240]:
for word in features.columns:
    features[word] = pd.to_numeric(features[word],errors = "coerce")

In [241]:

X_train, X_test, y_train, y_test = train_test_split(train, prediccion, test_size=0.30, random_state=41)

# Random Forest
# historial cambios tweaks | score

columnas: 
'ad campaign hit', 'brand listing', 'checkout', 'generic listing',
       'lead', 'search engine hit', 'searched products', 'staticpage',
       'viewed product', 'visited site', 'level_0', 'storage_no_convercion',
       'numero mas usado dia', 'condition_no_convercion2',
       'color_no_convercion2', 'search_engine2', 'nombre mas usado dia2']
       
features contraproductivos:
    cantidad de dias utilizado, hora/mes/dia ultimo evento, ultimo dia de la semana activo

test_size = 0.3
cv = 20
n_estimator 50 | .78 (Kaggle .76)
n_estimator 100 | .79 

cambio smootingbasado en: conversiones previas -> conversiones futuras

test_size = 0.3
cv = 20
n_estimator 100 | .80 (Kaggle .78)

agrego ultimo evento

test_size = 0.3
cv = 20
n_estimator 100 | .793


agrego: ultimo dias hasta el 1/6,suceso de cada evento, rellenados con -1

test_size = 0.3
cv = 20
n_estimator 100 | .798

agrego: ultimo dia de la semana activo

test_size = 0.3
cv = 20
n_estimator 100 | .798

cambio relleno de cantidad de veces de cada evento -1 -> mediana
agrego  'delta_ultimo_primer_dia_actividad',
       'delta_ultimo_anteulit_tiempo_actividad',
       'delta_primer_seg_tiempo_actividad',
       'dTimeEve

test_size = 0.3
cv = 20
n_estimator 100 | .832



https://www.youtube.com/watch?v=BSUMBBFjxrY

In [242]:
# probar clasificacion
randomforesttree = RandomForestRegressor(random_state = None, n_jobs=-1)
randomforesttree.get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [243]:
#
param_grid= {'max_features': ["auto"], 'n_estimators': [100], 'criterion': ['mse']}
grid_drop = GridSearchCV(randomforesttree, param_grid, cv=20, scoring='roc_auc',n_jobs = -1,pre_dispatch = 4)
grid_drop.fit(X_train, y_train)
print(grid_drop.best_params_, grid_drop.best_score_)

{'n_estimators': 100, 'criterion': 'mse', 'max_features': 'auto'} 0.8374812562766796


In [244]:
result = grid_drop.best_estimator_.predict(X_test)

In [245]:
roc_auc_score(y_test, result)

0.8351039424416593

In [246]:
feature_importances = pd.DataFrame(grid_drop.best_estimator_.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

In [247]:
feature_importances

,importance
checkout,0.077856
dHTimeEveMax,0.068522
checkout_last,0.052852
dHTimeEveMean1,0.052229
numero mas usado dia,0.050805
viewed product,0.049169
brand listing,0.043691
ultimo_evento_registrado_cmode,0.042386
Estado Bien,0.041819
color_no_convercion,0.040775


# Gradient Boost
# historial cambios tweaks | score 

columnas: 
'ad campaign hit', 'brand listing', 'checkout', 'generic listing',
       'lead', 'search engine hit', 'searched products', 'staticpage',
       'viewed product', 'visited site', 'level_0', 'storage_no_convercion',
       'numero mas usado dia', 'condition_no_convercion2',
       'color_no_convercion2', 'search_engine2', 'nombre mas usado dia2']


cambio smootingbasado en: conversiones previas -> conversiones futuras

agrego ultimo evento

agrego dias hasta el 1/6

test_size = 0.3
cv = 20
n_estimator 100 | .82 (Kaggle .815)


agrego: ultimo dia de semana

test_size = 0.3
cv = 20
n_estimator 100 | .820

agrego: ultimo suceso de cada evento, rellenados con -1

test_size = 0.3
cv = 20
n_estimator 100 | .824

agrego: cantidad de dias utilizado

test_size = 0.3
cv = 20
n_estimator 100 | .824



In [248]:
gradientboost = GradientBoostingRegressor(random_state = None)
gradientboost.get_params()

{'alpha': 0.9,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'presort': 'auto',
 'random_state': None,
 'subsample': 1.0,
 'verbose': 0,
 'warm_start': False}

In [249]:
param_grid= {'max_features': ["auto"], 'n_estimators': [30], 'loss': ['ls']}
grid_drop = GridSearchCV(gradientboost, param_grid, cv=20, scoring='roc_auc')
grid_drop.fit(X_train, y_train)
print(grid_drop.best_params_, grid_drop.best_score_)


{'loss': 'ls', 'n_estimators': 30, 'max_features': 'auto'} 0.8622493019373293


In [250]:
result2 = grid_drop.best_estimator_.predict(X_test)

In [251]:
roc_auc_score(y_test, result2)

0.8649306040782887

In [252]:
feature_importances2 = pd.DataFrame(grid_drop.best_estimator_.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances2

,importance
checkout,0.248857
checkout_last,0.164776
conversion_last,0.062467
brand listing,0.057694
viewed product_last,0.051491
conversion,0.039642
color_no_convercion,0.038825
Estado Muy Bien,0.038276
viewed product,0.031378
dHTimeEveMean1,0.028999


# Summit



In [253]:
to_pred = pd.read_csv("csv/trocafone_kaggle_test.csv")
features = pd.read_csv("features.csv")

In [254]:
to_pred_completo = pd.merge(to_pred,test,on="person",how = "left").dropna()

In [255]:
to_pred_completo["label"] = grid_drop.best_estimator_.predict(to_pred_completo.drop(["person"],axis = 1))

In [256]:
to_pred = pd.merge(to_pred_completo[["person","label"]],to_pred,on="person",how="right")

In [257]:
#to_pred["label"] = to_pred["label"].apply(richGetRicher)
to_pred["label"].loc[to_pred["label"] < 0] = 0
to_pred["label"].loc[to_pred["label"] > 1] = 1

c:\users\augusto joaquin\appdata\local\programs\python\python35\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [258]:
# mode < median < mean
to_pred = to_pred.fillna(to_pred["label"].mean())

In [259]:
to_pred.to_csv("sumit0.2", index=False)

In [136]:
def richGetRicher(x):
    if(x > .2):
        return x *10
    else: return x

In [34]:
to_pred["label"].isna().sum()

0

In [35]:
i

NameError: name 'i' is not defined